<a href="https://colab.research.google.com/github/fani-lab/OpeNTF/blob/main/ipynb/fair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`OpeNTF-Fair` via [`Adila`](https://github.com/fani-lab/Adila)

<p align="center"><img src='https://github.com/fani-lab/OpeNTF/blob/main/docs/figs/adila_.png?raw=true' width="400" ></p>

With [`Adila`](https://github.com/fani-lab/Adila)'s submodule in `OpeNTF`, we now have integrated fairness-aware reranking methods to rerank the final model's recommendation of experts for a team in order to dibiase `poularity` or `gender` disparities. To apply, `cmd=[..., fair]` and `fair.*` in [`./src/__config.yml`](https://github.com/fani-lab/OpeNTF/blob/main/src/__config__.yaml#L95C1-L110C24) should be set.

  - **fair**.`fgender`, contains the `female` member ids as the `minority` also `protected` group.

  > In `Adila`, we keep the labels of `minority` group for efficiency as they are very few relative to majority group. However, `protected` group could be the same as `minority` group, like in `gender` protected attribute, or the `majority` group (non-popular experts, who are more often than popular ones), like in `popularity` protected attribute.

  - **fair.**`algorithm`, list of reranking algorithms from `det_greedy`, `det_cons`, `det_relaxed`, `det_const_sort`, `fa-ir`
  - **fair.**`notion`, list of notions of fairness, from equality of odds (`eo`), demographic parity (`dp`)
  - **fair.**`attribute`, list of protected attributes from `popularity`, `gender`
  - **fair.**`is_popular_alg`, popularity status based on either `avg` teams per experts, i.e., whoever above this value is popular, or `auc` whoever is in the head of distribution figure, or both
  - **fair.**`k_max`, cutoff for the reranking algorithms
  - **fair.**`alpha`, the significance value for fa-ir algorithm
  - **fair.**`acceleration`, 'cpu' for `all` cpu cores but one, 'cpu:n' for `n` cores

```
fair:
  fgender: ../data/dblp/toy.dblp.v12.json.females.csv
  algorithm: [fa-ir, det_greedy, det_relaxed, det_const_sort, det_cons]
  notion: [eo,dp]
  attribute: [gender, popularity]
  is_popular_alg: [avg, auc]
  k_max: 5
  alpha: 0.1
  acceleration: 'cpu:1'
```


---


To **evaluate** the fair-accuracy trade-offs, fairness and accuracy metrics `before` and `after` applying reranking algorithms are calculated based on the `OpeNTF`'s evaluation settings in [`./src/__config.yml`](https://github.com/fani-lab/OpeNTF/blob/main/src/__config__.yaml#L84C1-L93C60)

- **eval.**`fair`, fairness measures `before` and `after` reranking using `ndkl` and `skew`
- **eval.**`topk`, cutoffs for measuring accuracy (utility) metrics
- **eval.**`trec`, measuring ranking-based accuracy (utility) metrics `before` and `after` reranking
- **eval.**`other`, measuring other utility metrics `before` and `after` reranking like `aucroc`, `skill_coverage`, ...
- **eval.**`per_epoch`, also apply rerankings on per-epoch predictions
- **eval.**`per_instance`, fairness and accuray metrics per each test team, needed for paired significance tests

```
eval:
  fair: [ndkl, skew]
  topk: '2,5,10'
  trec: [P_topk, recall_topk, ndcg_cut_topk, map_cut_topk, success_topk]
  other: [skill_coverage_topk, aucroc]
  per_epoch: 10
  per_instance: True
```

---


**Sample Command**
```
python main.py  "cmd=[prep,train,test,eval,fair]" \
                
                "models.instances=[mdl.rnd.Rnd]" \
                
                data.domain=cmn.publication.Publication \
                data.source=../data/dblp/toy.dblp.v12.json \
                data.output=../output/dblp/toy.dblp.v12.json \
                ~data.filter \
                
                fair.fgender=../data/dblp/toy.dblp.v12.json.females.csv \

```




**Setup & Quickstart**

From `OpeNTF`'s [`quickstart`](https://colab.research.google.com/github/fani-lab/OpeNTF/blob/main/ipynb/quickstart.ipynb) script:


In [1]:
# set up python 3.8
!sudo apt-get update -y
!sudo apt-get install -y python3.8 python3.8-venv python3.8-distutils python3-pip
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 10
!python --version

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://cli.github.com/packages stable InRelease [3,917 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,538 kB]
Get:12 https://cli.github.com/packages stable/main amd64 Packages [355 B]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [

In [2]:
# get OpeNTF and Adila submodule
!rm -R opentf/
!git clone --recurse-submodules https://github.com/Fani-Lab/opentf
!pip install --upgrade pip setuptools
!pip install -r opentf/requirements.txt

rm: cannot remove 'opentf/': No such file or directory
Cloning into 'opentf'...
remote: Enumerating objects: 27105, done.
remote: Counting objects: 100% (280/280), done.
remote: Compressing objects: 100% (213/213), done.
remote: Total 27105 (delta 125), reused 159 (delta 66), pack-reused 26825 (from 3)
Receiving objects: 100% (27105/27105), 1.32 GiB | 20.94 MiB/s, done.
Resolving deltas: 100% (13396/13396), done.
Updating files: 100% (4379/4379), done.
Submodule 'src/Adila' (https://github.com/fani-lab/Adila.git) registered for path 'src/Adila'
Cloning into '/content/opentf/src/Adila'...
remote: Enumerating objects: 1903, done.        
remote: Counting objects: 100% (155/155), done.        
remote: Compressing objects: 100% (93/93), done.        
remote: Total 1903 (delta 110), reused 69 (delta 62), pack-reused 1748 (from 1)        
Receiving objects: 100% (1903/1903), 17.64 MiB | 16.87 MiB/s, done.
Resolving deltas: 100% (1038/1038), done.
Submodule path 'src/Adila': checked out '708b

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.0/806.0 kB 18.4 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=438a7119e29e6bd9b878fefd5a09e21e224eddbbb3d236022fe006381c3736e2
  Stored in directory: /root/.cache/pip/wheels/b1/a3/c2/6df046c09459b73cc9bb6c4401b0be6c47048baf9a1617c485
Successfully built antlr4-python3-runtime
  Attempting uninstall: zipp
    Found existing installation: zipp 1.0.0
    Uninstalling zipp-1.0.0:
      Successfully uninstalled zipp-1.0.0


In [3]:
%cd opentf/src/
!python main.py  "cmd=[prep,train,test,eval,fair]" "models.instances=[mdl.rnd.Rnd]" data.domain=cmn.publication.Publication data.source=../data/dblp/toy.dblp.v12.json data.output=../output/dblp/toy.dblp.v12.json ~data.filter

/content/opentf/src
[2026-02-19 18:13:43,036][cmn.team][INFO] - Loading teamsvecs matrices from ../output/dblp/toy.dblp.v12.json/teamsvecs.pkl ...
[2026-02-19 18:13:43,039][pkgmgr][INFO] - tqdm not found.
[2026-02-19 18:13:43,039][pkgmgr][INFO] - Installing tqdm...
[2026-02-19 18:13:44,395][pkgmgr][INFO] - Collecting tqdm==4.65.0

[2026-02-19 18:13:44,400][cmn.team][INFO] - Loading indexes pickle from ../output/dblp/toy.dblp.v12.json/indexes.pkl ...
[2026-02-19 18:13:44,400][cmn.team][INFO] - Indexes pickle is loaded.
[2026-02-19 18:13:44,400][cmn.team][INFO] - Teamsvecs matrices and indexes for skills (31, 10), members (31, 13), and locations (31, 29) are loaded.
[2026-02-19 18:13:44,401][__main__][INFO] - Loading splits from ../output/dblp/toy.dblp.v12.json/splits.f3.r0.85.pkl ...
[2026-02-19 18:13:44,401][cmn.team][INFO] - Loading member-skill co-occurrence matrix (13, 10) from ../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/skillcoverage.pkl ...
[2026-02-19 18:13:44,437][pkgmgr][I

**Output Folders and Files**

From `Adila`'s [`quickstart`](https://colab.research.google.com/github/fani-lab/Adila/blob/main/quickstart.ipynb) script:

- `adila/{popularity,gender}/stats.pkl`, the distribution of popular/non-popular, or male/female experts in the entire dataset
- `adila/{popularity,gender}/labels.csv`, the ids for popular or female experts. While the female ids are fixed and obtained from the `data.fgender` file, the popular ids depends on `is_popular_alg` and is calculated based on the data distribution

- `adila/{popularity,gender}/{notion}`, the subfolder containg the results based on the fairness notion
- `adila/{popularity,gender}/{notion}/{data.fpred}.{fair.algorithm}.{fair.k_max}.rerank.pred`, the reranked version of the recommended members from `data.fpred`
- `adila/{popularity,gender}/{notion}/{data.fpred}.{fair.algorithm}.{fair.k_max}.rerank.pred.eval.fair.instance.csv`, the fairness metric values for `data.fpred` (before) and the reranked version (after) per each team instance in the test set  
- `adila/{popularity,gender}/{notion}/{data.fpred}.{fair.algorithm}.{fair.k_max}.rerank.pred.eval.fair.mean.csv`, the average of fairness metric values for `data.fpred` (before) and the reranked version (after) over all teams of the test set  
- `adila/{popularity,gender}/{notion}/{data.fpred}.{fair.algorithm}.{fair.k_max}.rerank.pred.eval.utility.instance.csv`, the accuracy metric values for `data.fpred` (before) and the reranked version (after) per each team instance in the test set  
- `adila/{popularity,gender}/{notion}/{data.fpred}.{fair.algorithm}.{fair.k_max}.rerank.pred.eval.utility.mean.csv`, the average of accuracy metric values for `data.fpred` (before) and the reranked version (after) over all teams of the test set

In [4]:
!find ../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000/adila/ -print | sed 's;[^/]*/;│   ;g;s;│   \([^│]\);├── \1;'

│   │   │   │   │   │   │   
│   │   │   │   │   │   ├── popularity.auc
│   │   │   │   │   │   │   ├── stats.pkl
│   │   │   │   │   │   │   ├── dp
│   │   │   │   │   │   │   │   ├── f0.test.pred.det_greedy.auc.5.rerank.pred.eval.fair.mean.csv
│   │   │   │   │   │   │   │   ├── f1.test.pred.det_relaxed.auc.5.rerank.pred
│   │   │   │   │   │   │   │   ├── f1.test.pred.det_const_sort.auc.5.rerank.pred.eval.fair.mean.csv
│   │   │   │   │   │   │   │   ├── f1.test.pred.det_const_sort.auc.5.rerank.pred.eval.utility.mean.csv
│   │   │   │   │   │   │   │   ├── f1.test.pred.det_cons.auc.5.rerank.pred.eval.fair.instance.csv
│   │   │   │   │   │   │   │   ├── f1.test.pred.det_relaxed.auc.5.rerank.pred.eval.utility.instance.csv
│   │   │   │   │   │   │   │   ├── f2.test.pred.det_cons.auc.5.rerank.pred.eval.utility.mean.csv
│   │   │   │   │   │   │   │   ├── f1.test.pred.det_const_sort.auc.5.rerank.pred.eval.utility.instance.csv
│   │   │   │   │   │   │   │   ├── f1.test.pred.fa-ir.auc.1

**Average fairness measures**






In [7]:
import pandas as pd
pd.read_csv('../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000/adila/popularity.avg/eo/f0.test.pred.fa-ir.avg.10.5.rerank.pred.eval.fair.mean.csv', index_col=0)

,mean.after,mean.before
metrics,,
ndkl,0.171038,0.171038
skew.majority,0.025318,0.025318
skew.minority,-0.262364,-0.262364


**Fairness measures per each test team**

In [8]:
import pandas as pd
pd.read_csv('../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000/adila/popularity.avg/eo/f0.test.pred.fa-ir.avg.10.5.rerank.pred.eval.fair.instance.csv')

,before.ndkl,after.ndkl,before.skew.minority,before.skew.majority,after.skew.minority,after.skew.majority
0,0.100350,0.100350,-0.262364,0.025318,-0.262364,0.025318
1,0.066075,0.066075,-0.262364,0.025318,-0.262364,0.025318
2,0.055013,0.055013,-0.262364,0.025318,-0.262364,0.025318
3,0.522018,0.522018,-0.262364,0.025318,-0.262364,0.025318
4,0.111735,0.111735,-0.262364,0.025318,-0.262364,0.025318


**Average accuracy values (Utility)**


In [9]:
import pandas as pd
pd.read_csv('../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000/adila/popularity.avg/eo/f0.test.pred.fa-ir.avg.10.5.rerank.pred.eval.utility.mean.csv', index_col = 0)

,mean.before,mean.after
metric,,
P_2,0.200000,0.200000
P_5,0.120000,0.120000
P_10,0.140000,0.140000
recall_2,0.200000,0.200000
recall_5,0.300000,0.300000
recall_10,0.666667,0.666667
ndcg_cut_2,0.200000,0.200000
ndcg_cut_5,0.252814,0.252814
ndcg_cut_10,0.400669,0.400669


**Accuracy values per each test team (Utility)**


In [10]:
import pandas as pd
pd.read_csv('../output/dblp/toy.dblp.v12.json/splits.f3.r0.85/rnd.b1000/adila/popularity.avg/eo/f0.test.pred.fa-ir.avg.10.5.rerank.pred.eval.utility.instance.csv')

,P_2.before,P_5.before,P_10.before,recall_2.before,recall_5.before,recall_10.before,ndcg_cut_2.before,ndcg_cut_5.before,ndcg_cut_10.before,map_cut_2.before,...,ndcg_cut_10.after,map_cut_2.after,map_cut_5.after,map_cut_10.after,success_2.after,success_5.after,success_10.after,skill_coverage_2.after,skill_coverage_5.after,skill_coverage_10.after
0,0.0,0.0,0.1,0.0,0.0,0.50000,0.00000,0.00000,0.19343,0.00,...,0.19343,0.00,0.000,0.06250,0.0,0.0,1.0,1.0,1.0,1.0
1,0.5,0.2,0.2,0.5,0.5,1.00000,0.61315,0.61315,0.81753,0.50,...,0.81753,0.50,0.500,0.64286,1.0,1.0,1.0,1.0,1.0,1.0
2,0.0,0.0,0.1,0.0,0.0,0.33333,0.00000,0.00000,0.14804,0.00,...,0.14804,0.00,0.000,0.04167,0.0,0.0,1.0,1.0,1.0,1.0
3,0.0,0.2,0.1,0.0,0.5,0.50000,0.00000,0.26407,0.26407,0.00,...,0.26407,0.00,0.125,0.12500,0.0,1.0,1.0,1.0,1.0,1.0
4,0.5,0.2,0.2,0.5,0.5,1.00000,0.38685,0.38685,0.58028,0.25,...,0.58028,0.25,0.250,0.37500,1.0,1.0,1.0,1.0,1.0,1.0
